# Simulação de regressão linear em dados rotulados

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

24/01/02 17:45:09 WARN Utils: Your hostname, Valhalla-Zorin resolves to a loopback address: 127.0.1.1; using 192.168.100.36 instead (on interface enp4s0)
24/01/02 17:45:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/02 17:45:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
data = spark.read.csv('Data/Ecommerce_Customers.csv', inferSchema=True, header=True)

In [5]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [6]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [15]:
selectedColumns = data.columns[3:7]
selectedColumns

['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership']

In [16]:
assembler = VectorAssembler(inputCols=selectedColumns, outputCol='features')

In [17]:
output = assembler.transform(data)

In [18]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [19]:
final_data = output.select('features', 'Yearly Amount Spent')

In [20]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [21]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [24]:
train_data.describe().show(), test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                341|
|   mean|  502.3897186706369|
| stddev|  84.24971747404958|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                159|
|   mean| 492.71776768935894|
| stddev|  67.30138982144778|
|    min|  282.4712457199145|
|    max|  663.8036932755083|
+-------+-------------------+



(None, None)

In [25]:
lr = LinearRegression(labelCol='Yearly Amount Spent') # O nome da coluna de features já é igual ao default da função

In [26]:
lr_model = lr.fit(train_data)

24/01/02 17:57:21 WARN Instrumentation: [9339706f] regParam is zero, which might cause numerical instability and overfitting.
24/01/02 17:57:21 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/01/02 17:57:21 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/01/02 17:57:21 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [27]:
test_results = lr_model.evaluate(test_data)

In [28]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -5.168160184040119|
| -4.174408975562699|
| -22.39181965455907|
| -7.522617922827294|
| -4.751132744401048|
|  20.49525390932473|
| 3.2455202728262407|
| -4.958313003711396|
|   4.41380131890952|
|-2.0293084085816417|
| 16.345441204578094|
|-18.892279893507123|
| -9.098969813143583|
|  7.482007629641259|
|-11.043770559659265|
|  12.67637512623702|
|-13.464225504686283|
| 0.7997497583177733|
| 19.322612377943415|
|  5.440210369920919|
+-------------------+
only showing top 20 rows



In [29]:
test_results.rootMeanSquaredError

10.578303525614292

In [30]:
test_results.r2

0.9751386893586445

In [31]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



# Aplicando em dados não rotulados

In [32]:
unlabeled_data = test_data.select('features')

In [33]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.4925366965402...|
|[30.8794843441274...|
|[31.1239743499119...|
|[31.1280900496166...|
|[31.2681042107507...|
|[31.3123495994443...|
|[31.4459724827577...|
|[31.5257524169682...|
|[31.5316044825729...|
|[31.5761319713222...|
|[31.6098395733896...|
|[31.8164283341993...|
|[31.8279790554652...|
|[31.8512531286083...|
|[31.8648325480987...|
|[31.9262720263601...|
|[31.9365486184489...|
|[32.0047530203648...|
|[32.0180740106320...|
|[32.0478009788678...|
+--------------------+
only showing top 20 rows



In [34]:
predictions = lr_model.transform(unlabeled_data)

In [35]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...|287.63940590395464|
|[30.8794843441274...| 494.3810089604174|
|[31.1239743499119...|509.33887349432484|
|[31.1280900496166...|  564.775304669882|
|[31.2681042107507...|428.22166591822497|
|[31.3123495994443...| 443.0961641186159|
|[31.4459724827577...|481.63144466230233|
|[31.5257524169682...| 448.9239398135933|
|[31.5316044825729...|432.10180441045304|
|[31.5761319713222...|   543.25589239791|
|[31.6098395733896...|428.20010844653007|
|[31.8164283341993...| 520.0147713971635|
|[31.8279790554652...| 449.1017173600851|
|[31.8512531286083...|465.51023903715713|
|[31.8648325480987...|450.93505103647294|
|[31.9262720263601...| 379.5285583180894|
|[31.9365486184489...| 440.6636104000145|
|[32.0047530203648...|462.94623136231166|
|[32.0180740106320...| 338.4604983673719|
|[32.0478009788678...|508.01036081617553|
+--------------------+------------